# Role R Likelihood-free Simulation Based Inference

In [ ]:
from roler.model import ModelPrior
from roler.distributions import *

## Defining the Model Prior

This cell defines the prior distributions and fixed values for the simulation model using the `ModelPrior` class. This object will be used to sample parameter values for each simulation run.

In [7]:
prior = ModelPrior(
    individuals_local=IntDistribution(50, 300),
    individuals_meta=IntDistribution(400, 1000),
    species_meta=50,
    speciation_local=0.05,
    speciation_meta=0.05,
    extinction_meta=0.05,
    env_sigma=0.5,
    trait_sigma=1,
    comp_sigma=0.5,
    dispersal_prob=0.1,
    mutation_rate=0.01,
    equilib_escape=1,
    num_basepairs=250,
    init_type='oceanic_island',
    niter=2000,
    niterTimestep=10
)

## Running a Single Simulation and Extracting Summary Statistics

This cell executes a single simulation run. First, it initializes a `Simulator` object. Then, it samples a set of parameter values (`theta`) from the `prior` distribution defined in the previous cell. Finally, it runs the simulation using these sampled parameters and stores the resulting summary statistics in the `stats` variable. The `stats` variable is then printed to display the simulation output.

In [8]:
from roler.simulation import Simulator

simulator = Simulator()
theta = prior.sample()
stats = simulator.simulate(theta)
stats

,hill_abund_1,hill_abund_2,hill_abund_3,hill_abund_4,hill_gen_1,hill_gen_2,hill_gen_3,hill_gen_4,hill_trait_1,hill_trait_2,hill_trait_3,hill_trait_4,hill_phy_1,hill_phy_2,hill_phy_3,hill_phy_4,richness,iteration
1,1.000000,1.000000,1.000000,1.000000,1.0,inf,inf,inf,NaN,NaN,NaN,NaN,1,2,3,4,1,0.0
2,1.124583,1.039993,1.029948,1.026577,1.0,inf,inf,inf,12.852082,12.595326,12.564004,12.553442,1,2,3,4,4,10.0
3,1.264197,1.082428,1.061422,1.054416,1.0,inf,inf,inf,13.486389,12.821148,12.740232,12.713006,1,2,3,4,7,20.0
4,1.264197,1.082428,1.061422,1.054416,1.0,inf,inf,inf,13.309841,12.786839,12.723056,12.701577,1,2,3,4,7,30.0
5,1.264197,1.082428,1.061422,1.054416,1.0,inf,inf,inf,13.442307,12.812598,12.735954,12.710160,1,2,3,4,7,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,18.815066,11.977778,9.371015,8.133974,1.0,inf,inf,inf,22.698186,14.993130,11.868148,10.319213,1,2,3,4,35,1960.0
198,16.908295,10.939114,8.797063,7.751635,1.0,inf,inf,inf,19.878195,13.008964,10.566588,9.424357,1,2,3,4,33,1970.0
199,17.680304,11.740594,9.471291,8.341453,1.0,inf,inf,inf,20.259220,13.627678,11.140963,9.931639,1,2,3,4,33,1980.0
200,18.462503,12.162051,9.659610,8.422031,1.0,inf,inf,inf,20.990121,14.179081,11.521312,10.199314,1,2,3,4,34,1990.0


## Generating a Dataset of Simulations

This cell generates a dataset of simulation results using the `Dataset` class. It initializes a `Dataset` object with the previously defined `simulator` and `prior`. The `generate_dataset` method is then called to generate `samples=10` simulation runs. The resulting parameter values (`theta`) and summary statistics (`x`) are stored. Finally, the shapes of `theta` and `x` are printed to show the dimensions of the generated dataset.

In [11]:
from roler.datasets import Dataset

dataset = Dataset(simulator=simulator, prior=prior)
theta, x = dataset.generate_dataset(samples=10)

print("theta.size", theta.shape)
print("x.size", x.shape)

theta.size torch.Size([1995, 2])
x.size torch.Size([1995, 9])


## Performing Inference with SNPE (Sequential Neural Posterior Estimation)

This cell uses the `sbi` library to perform inference and estimate the posterior distribution of the model parameters given the simulated data.

1.  An `SNPE` object is initialized using the joint uniform prior obtained from `prior.get_joint_uniform()`.
2.  The simulated parameter values (`theta`) and summary statistics (`x`) generated in the previous cell are used to train a neural density estimator using `snpe.append_simulations(theta, x).train()`.
3.  A posterior object is built from the trained density estimator using `snpe.build_posterior(density_estimator)`. This posterior object can then be used to sample from the approximate posterior distribution or evaluate its density.

In [12]:
from sbi.inference import SNPE

snpe = SNPE(prior=prior.get_joint_uniform())
density_estimator = snpe.append_simulations(theta, x).train()
posterior = snpe.build_posterior(density_estimator)

 Neural network successfully converged after 362 epochs.

## Evaluating the Learned Posterior with Observed Data

This cell evaluates the performance of the learned posterior by:

1.  Generating a single "observed" dataset point `x_obs` with corresponding true parameter values `theta_true` using `dataset.generate_dataset(samples=1)`.
2.  Sampling from the posterior distribution conditioned on the observed data `x_obs` using `posterior.sample((1000,), x=x_obs)`, creating 1000 posterior samples.
3.  Computing the posterior mean as a point estimate of the parameters.
4.  Printing the true parameter values (`theta_true`) and the posterior mean estimate for comparison. This allows you to assess how well the inference procedure recovers the true parameters given the observed data.

In [ ]:
theta_true, x_obs = dataset.generate_dataset(samples=1)
theta_true, x_obs = theta_true[-1], x_obs[-1]
print("Observed simulation output:", x_obs)

# Use the learned posterior to sample inferred parameters given the observed output
posterior_samples = posterior.sample((1000,), x=x_obs)
print("Posterior samples shape:", posterior_samples.shape)

# Compute a point estimate (e.g. the posterior mean)
posterior_mean = posterior_samples.mean(dim=0)
print("Posterior mean estimate:", posterior_mean)

print()
print("Theta True      :", theta_true)
print("Theta Prediction:", posterior_mean)

Observed simulation output: tensor([18.9147, 10.8582,  7.6717,  6.3535, 20.6015, 13.7787, 11.2295, 10.0382,
        33.0000])


Drawing 1000 posterior samples:   0%|          | 0/1000 [00:00<?, ?it/s]

Posterior samples shape: torch.Size([1000, 2])
Posterior mean estimate: tensor([165.3517, 560.4550])

Theta True      : tensor([125.0480, 806.7888])
Theta Prediction: tensor([165.3517, 560.4550])


In [ ]:
print("True       :", dataset.get_params_from_tensor(theta_true))
print("Prediction :", dataset.get_params_from_tensor(posterior_mean))

True       : ModelParams(individuals_local=125, individuals_meta=807, species_meta=50, speciation_local=0.05, speciation_meta=0.05, extinction_meta=0.05, env_sigma=0.5, trait_sigma=1, comp_sigma=0.5, dispersal_prob=0.1, mutation_rate=0.01, equilib_escape=1, num_basepairs=250, init_type='oceanic_island', niter=2000, niterTimestep=10)
Prediction : ModelParams(individuals_local=165, individuals_meta=560, species_meta=50, speciation_local=0.05, speciation_meta=0.05, extinction_meta=0.05, env_sigma=0.5, trait_sigma=1, comp_sigma=0.5, dispersal_prob=0.1, mutation_rate=0.01, equilib_escape=1, num_basepairs=250, init_type='oceanic_island', niter=2000, niterTimestep=10)
